#### **Worldwide Earthquake Events API- Gold Layer Processing**

In [35]:
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import when
# ensure the below library is installed on your fabric environment
import reverse_geocoder as rg


StatementMeta(, 7804467d-2057-411f-b9fb-af9093664739, 5, Finished, Available, Finished)

In [2]:
df = spark.read.table("earthquake_events_silver").filter(col('time')> start_date)

StatementMeta(, 7804467d-2057-411f-b9fb-af9093664739, 6, Finished, Available, Finished)

In [9]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): latitude of the location
    lon (float or str): longitude of the location

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API

    Example:
    >>> get_country_code(36.0088333, -117.918)
    'US'
    
    """
    coordinates = (float(lat), float(lon))
    return rg.search(coordinates)[0].get('cc')

StatementMeta(, 7804467d-2057-411f-b9fb-af9093664739, 7, Finished, Available, Finished)

In [15]:
# registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(, 7804467d-2057-411f-b9fb-af9093664739, 8, Finished, Available, Finished)

In [17]:
# adding country_code and city attributes
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col('latitude'), col('longitude')))

StatementMeta(, 7804467d-2057-411f-b9fb-af9093664739, 9, Finished, Available, Finished)

In [29]:
# adding significance classification
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class',
                                            when(col("significant") < 100, "Low").\
                                            when((col("significant") >= 100) & (col("significant") < 500), "Moderate").\
                                            otherwise("High")
                                            )
                                            

StatementMeta(, 7804467d-2057-411f-b9fb-af9093664739, 10, Finished, Available, Finished)

In [ ]:
# appending the data to the gold table
df_with_location_sig_class.write.mode('append').saveAsTable('earthquake_events_gold')